# Formula Student EV

In [50]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import numpy as np
from IPython.display import display
%matplotlib inline

## Important takes from 2021 FS Rules
- Battery can output a maximum of 80kW (107HP).
- Regen is allowed
- Overcurrent Protection is required. (Main Fuse)
- A maximum 600V DC is allowed. And 630V DC when considering signal. 
- All components in the traction system must be rated for the maximum voltage.
- HVIL is allowed. But HV and LV connections are not allowed on the same connector. 
- All HV components must be inside the roll structure, exception are hub motors. 
- All HV components must be protected from side or rear impacts. 
- HV and LV are allowed on the same PCB, isolation devices (opto-isolators) may be used and established component spacing is required. 
- Electrical tape and rubber like coating is not allowed on TS components. 
- TS components must be waterproof
- External HV cabling must have a grounded shield .
- External HV crimps must withstand 200N. 
- All electrical connections must have positive locking mechanisms.
- Soldered connections are not allowed, except for SMD devices. 
- An event provided data logger must be used to log voltage and current. 
- 2 Tractive System Measuring Points must be placed next to the master switches. 
- Battery to Inverter connections must be external and removable without tools and above 350mm from the ground. 
- Inverter must have an active discharge circuit that will drain voltage in the system when a disconnect is detected or power is turned off. 
- Battery modules must not exceed 120V, not be over 1.6kWh and weigh less than 12kg. 
- Interchangeable batteries are allowed at the competitions. 
- DCDC Converters are allowed inside the battery box
- Battery modules must be separated by isolating/fire retardant material. 
- The battery box cannot be part of the impact structure.
- The battery box may be made out of any material, materials other than steel or aluminium they must be submitted in the SES. 
- The battery pack requires a pressure release valve. 
- Pouch cell tabs must not carry any mechanical load. 
- A removable temperature logging device must be used. 
- Sensing Pre-charge Circuit is required. 
- The BMS must measure:
    - Cell voltages
    - Current
    - Temperature of warmest cells 
    - Temperature of at least 30% of the cells, equally distributed. 
- Cell temperature must be measured on the negative terminal of the cell. 
- **Battery modules must be disconnected without tools?**
- An insulation fault detection device must be used. 
- Chargers must be presented at the technical inspection.
- Team may build their own chargers. 
- Software Calibration and Charging are allowed after technical inspection. 
- The battery must be able to be disassembled at the technical inspection. 
- The car is submitted to a rain test. Comparable IP rating would be IP65.


## Battery Specs
- For rear only motors Regen is around 30% of consumed energy. Close to 40% for 4wd. 
- Most teams calculate from previous telemetry that consumption is 9kWh during endurance. 
- Accounting for Regen most teams use battery packs between 6kWh and 6.5kWh. 
- ~30% of teams use pouch based cells, this percentage is higher for European Teams. Most teams use cylindrical 18650 cells. 
- Inside cylindrical cells most teams use Sony VTC6 for availability and higher energy density. 
- I personally have used Samsung 30T 21700 cells and they have very low internal resistance. 


### Cell Data

In [97]:
# Cell data indexed as 
dataind = {"CN":"Cell Model","RC":"Rated Capacity (mAh)","MV":"Max Voltage (V)","NV":"Nominal Voltage (V)",
           "IR":"Internal Resistance (mOhm)","CD":"Continous Discharge (A)", "W":"Weight (g)"}

cells = [{"CN":"Sony VTC6 18650", "RC":3000, "MV":4.2, "NV":3.6, "IR":21.4,"W":46.9,"CD":15},
          {"CN":"Samsung 30T 21700", "RC":3000, "MV":4.2, "NV":3.6, "IR":11.6,"W":68.1,"CD":35}]

cellFrame = pd.DataFrame.from_records(cells).transpose().rename(index = dataind)
cellFrame.rename(columns= cellFrame.iloc[0]).drop(cellFrame.index[0])



,Sony VTC6 18650,Samsung 30T 21700
Rated Capacity (mAh),3000,3000
Max Voltage (V),4.2,4.2
Nominal Voltage (V),3.6,3.6
Internal Resistance (mOhm),21.4,11.6
Weight (g),46.9,68.1
Continous Discharge (A),15,35


### Calculated Cell Data

In [111]:
@widgets.interact(NS=widgets.IntSlider(min=100, max=150, step=1, value=142, description="Series"),NP=widgets.IntSlider(min=1, max=6, step=1, value=4, description="Parallels"))
def f(NS,NP):
    tempval = pd.DataFrame(index=list(cellFrame),columns=['Total Number of cells','Nominal Capacity (kWh)','Max Pack Voltage(V)','Nominal Pack Voltage(V)',
                                                          'Series Current for 80kW @ Vnom (A)','Max Continous Power @ Vnom (kW)','Weight of the pack (kg)'])
    for i in range(len(cellFrame.columns)):
        NPV = cellFrame.loc[dataind['NV']][i]*NS # Nominal Pack Voltage (V)
        MPV = cellFrame.loc[dataind['MV']][i]*NS # Max Pack Voltage (V)
        NPC = cellFrame.loc[dataind['RC']][i]*NP*NPV/10**6 # Nominal Pack Capacity (kWh)
        C80kW = 80000/NPV/NP # Series Current for 80kW @ Nominal Voltage (A)
        MCP = cellFrame.loc[dataind['CD']][i]*NPV*NP/1000 # Max Continous Power @ Vnom (kW)
        PW = cellFrame.loc[dataind['W']][i]*NS*NP/1000 # Pack Weight (kg)
        tempval.iloc[i] = [NS*NP,NPC,MPV,NPV,C80kW,MCP,PW]
        
    return tempval.transpose()

interactive(children=(IntSlider(value=142, description='Series', max=150, min=100), IntSlider(value=4, descrip…